In [1]:
# script for analyzing sensitivity results

In [2]:
import os
import sys
import copy
import pickle
import concurrent.futures
import numpy as np
import pandas as pd

import cantera as ct
import rmgpy.chemkin

sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/kinetics')
import kineticfun


import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model'
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/naive_rmg_model'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)


438749050.py:8 <module> INFO Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


In [4]:
# base_delay = 0.14857535  # /work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model -- original 24 hour model
base_delay = 0.006115998233733183  # /work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model -- naive 24 hour model



In [5]:
# load the csv to get the ignition delay times
# csv_file = '/scratch/harris.se/autoscience/local_sensitivity/practice.csv'
# csv_file = '/scratch/harris.se/autoscience/local_sensitivity/ignition_delays_007.csv'
csv_file = '/scratch/harris.se/autoscience/naive_model/sensitivity/results/ignition_delays_0007.csv'
df = pd.read_csv(csv_file, skiprows=1, names=['model', 'delay'])

In [6]:
# get the parameters that make the biggest difference
delays = df['delay'].values
for i in range(0, len(delays)):
    if delays[i] <= 0:
        delays[i] = base_delay

        
order = np.arange(0, len(delays))
differences = np.abs(delays - base_delay)
most_sensitive = [x for _, x in sorted(zip(differences, order))][::-1]

In [7]:
most_sensitive[1]

108

In [8]:
# print the top parameters


# lookup up manually:
# 87 -> 313
# 77 -> 288
# 34 -> 247

for i in range(0, 500):
    index = most_sensitive[i]
    if index > len(reaction_list):
        index = index - len(reaction_list)
        print(index, '\t', species_list[index])
    else:
        try:
            print(index, '\t', reaction_list[index], '\t', reaction_list[index].family)
        except AttributeError:
            print(index, '\t', reaction_list[index], '\t', 'PDEP')

87 	 OH(15) + C2H(4) <=> CO(6) + CH2(23) 	 combustion_core/version5
108 	 HO2(16) + CH3CO(20) <=> OH(15) + CO2(7) + CH3(18) 	 FFCM1(-)
55 	 OH(15) + CH(3) <=> H(14) + HCO(19) 	 FFCM1(-)
77 	 O2(2) + C2H(4) <=> CO(6) + HCO(19) 	 FFCM1(-)
37 	 O(5) + C2H4(11) <=> H2O(8) + C2H2(25) 	 C2H4+O_Klipp2017
34 	 O(5) + C2H4(11) <=> CH2(23) + CH2O(9) 	 C2H4+O_Klipp2017
54 	 O2(2) + C2H2(25) <=> HCO(19) + HCO(19) 	 Klippenstein_Glarborg2016
123 	 OH(15) + CH2(23) <=> H(14) + CH2O(9) 	 FFCM1(-)
306 	 C4H8OOH1-3(219) => OH(15) + CH2O(9) + C3H6(12) 	 CurranPentane
24 	 H(14) + HO2(16) <=> O(5) + H2O(8) 	 BurkeH2O2inN2
485 	 C2H(4) + IC3H7(93) <=> C2H2(25) + C3H6(12) 	 Disproportionation
410 	 CH3(18) + PC4H9(182) <=> CH4(10) + C4H8-1(188) 	 Disproportionation
74 	 O(5) + CH2(23) <=> H(14) + H(14) + CO(6) 	 FFCM1(-)
305 	 PC4H9O2(184) <=> C4H8OOH1-3(219) 	 CurranPentane
326 	 C4H8OOH1-3O2(225) <=> C4H71-1,3OOH(238) 	 CurranPentane
420 	 HO2(16) + SC4H9(183) <=> H2O2(17) + C4H8-2(189) 	 Disproportionat

In [9]:
# convert the reaction numbers to primary model numbers for AutoTST calculation
for i in range(0, 50):
    rxn_smiles = kineticfun.reaction2smiles(reaction_list[most_sensitive[i]])
    primary_model_index = kineticfun.reaction_smiles2index(rxn_smiles)
    print(most_sensitive[i], '\t', primary_model_index, reaction_list[most_sensitive[i]], )


87 	 87 OH(15) + C2H(4) <=> CO(6) + CH2(23)
108 	 108 HO2(16) + CH3CO(20) <=> OH(15) + CO2(7) + CH3(18)
55 	 55 OH(15) + CH(3) <=> H(14) + HCO(19)
77 	 77 O2(2) + C2H(4) <=> CO(6) + HCO(19)
37 	 37 O(5) + C2H4(11) <=> H2O(8) + C2H2(25)
34 	 34 O(5) + C2H4(11) <=> CH2(23) + CH2O(9)
54 	 54 O2(2) + C2H2(25) <=> HCO(19) + HCO(19)
123 	 123 OH(15) + CH2(23) <=> H(14) + CH2O(9)
306 	 306 C4H8OOH1-3(219) => OH(15) + CH2O(9) + C3H6(12)
24 	 24 H(14) + HO2(16) <=> O(5) + H2O(8)
485 	 485 C2H(4) + IC3H7(93) <=> C2H2(25) + C3H6(12)
410 	 410 CH3(18) + PC4H9(182) <=> CH4(10) + C4H8-1(188)
74 	 74 O(5) + CH2(23) <=> H(14) + H(14) + CO(6)


NameError: name 'np' is not defined

In [ ]:
for i in range(0, len(delays)):
    if most_sensitive[i] > len(reaction_list):
        sp_index = most_sensitive[i] - len(reaction_list)
        print(sp_index, '\t', species_list[sp_index])

In [ ]:
len(reaction_list) + len(species_list)